# Human Standing Control Parameter Identification with Direct Collocation

Ton van den Bogert  
Jason K. Moore
<img width=300px src="NewCSU-stacked.svg"/>


July 10, 2015  
ISB TGCS 2015, Edinburgh, UK

# Parameter Identification Issues

- Nonlinear (and some linear) systems may require computationally intensive minimization methods
- Local minima are typical (requires superb guess)
- Systems may be unstable

# Local minima: Simple pendulum

<img style="float:right;" src="pendulum-objective.png" width=400px>

## 1-DoF, 1 parameter pendulum equations of motion

$\dot{\mathbf{x}} = \begin{bmatrix} \dot{\theta}(t) \\ \dot{\omega}(t) \end{bmatrix} = \begin{bmatrix} \omega(t) \\ -p \sin{\theta}(t) \end{bmatrix}$

## Objective: Minimize least squares

$J(p) = \min\limits_{p} \int_{t_0}^{t_f} [\theta_m(t) - \theta(\mathbf{x}, p, t)]^2 dt$

## References
> Vyasarayani, Chandrika P., Thomas Uchida, Ashwin Carvalho, and John McPhee.
"Parameter Identification in Dynamic Systems Using the Homotopy Optimization
Approach". Multibody System Dynamics 26, no. 4 (2011): 411-24.

# Direct Collocation

> Betts, J., and W. Huffman. “Large Scale Parameter Estimation Using Sparse Nonlinear Programming Methods.” SIAM Journal on Optimization 14, no. 1 (January 1, 2003): 223–44. doi:10.1137/S1052623401399216.

## Benefits

- Fast computation times
- Handles unstable systems with ease
- Less susceptible to local minima

## Disadvantages

- Exact solution requires large number of nodes
- Tedious and error prone to form sparse gradients, Jacobians, and Hessians

# Our Direct Collocation Implementation

## First order discrete integration options:

- Backward Euler: $\frac{x_i - x_{i + 1}}{h} = f(x_i, t_i)$
- Midpoint Rule: $\frac{x_{i + 1} - x_i}{h} = f(\frac{x_i + x_{i + 1}}{2}, t_i)$

## Implicit Continous Equations of Motion
 
$$0 = \mathbf{f}(\dot{\mathbf{x}}, \mathbf{x}, \mathbf{r}, \mathbf{p}, t)$$

## Nonlinear Programming Formulation
 
$$\min\limits_{\theta} J(\theta) \textrm{ where } \mathbf{\theta} = [\mathbf{x}, \mathbf{r}, \mathbf{p}]$$

$$\textrm{subject to } g(\theta) = \mathbf{f}(\dot{\mathbf{x}}, \mathbf{x}, \mathbf{r}, \mathbf{p}, t) =0 \textrm{ and } \theta_L \leq \theta \leq \theta_U$$

# Software Tool: opty

- User specifies continous symbolic:
  - objective
  - equations of motion
  - additional constraints
- Effficient just-in-time compiled C code is generated for functions that evaluate:
  - objective and its gradient
  - constraints and its Jacobian
- NLP problem automatically formed for IPOPT
- Open source: BSD license
- Written in Python
- http://github.com/csu-hmc/opty


# Example: 1 DoF, 1 Parameter Pendulum
Paraphrased from https://github.com/csu-hmc/opty/blob/master/examples/vyasarayani2011.py
```python
# Specify symbols for the parameters
p, t = symbols('p, t')

# Specify the functions of time
theta, omega, theta_m = symbols('theta, omega, theta_m', cls=Function)

# Specify the symbolic equations of motion
eom = (Matrix([theta(t), omega(t)]).diff(t) - 
       Matrix([omega(t), -p * sin(theta(t))]))

# Specify the symbolic objective function
obj = Integral((theta_m(t) - theta(t))**2, t)

# Choose discritzation values
num_nodes = 1000
interval = 0.01  # seconds

# Form the problem
prob = Problem(obj, eom, (theta(t), omega(t)), num_nodes, interval,
               known_trajectory_map={y1_m(t): measured_data},
               integration_method='midpoint')

# Set and initial guess
initial_guess = random(prob.num_free)

# Solve the system
solution, info = prob.solve(initial_guess)
```

# Computational Speed

## Example Larger System

- 11 DoF, 22 states, 22 parameters, 11 exogenous inputs
- 12840 mathematical operations in constraint expressions

## Discretization Variables

- 10,000 collocation nodes
- 219,978 constraints
- 16,938,306 nonzero entries in the Jacobian
- 330,022 free variables

## Timings

- Integrating with ODEPACK lsoda: **49.8 s**
- Constraint evaluation: **25.9 ms (0.0259 s)**
- Jacobian evaluation: **95.7 ms (0.0957 s)**

# Case Study: Human Control Parameter Identification

<div>
  <h2>Plant</h2>
  <img style="float:right;margin=5px;" src="free-body-diagram.svg" width=200px>
  <ul style="display: inline;">
    <li>Torque driven two-link inverted pendulum with an accelerating base.</li> 
    <li>States: $\mathbf{x}=[\theta_a \quad \theta_h \quad \omega_a \quad \omega_h]^T$</li>
    <li>Exogoneous inputs:
      <ul>
        <li>Controlled: $\mathbf{r}_c = [T_a \quad T_h]^T$</li>
        <li>Specified: $\mathbf{r}_k = [a]$</li>
      </ul>
    </li>
    <li>Known parameters: $\mathbf{p}_k$</li>
  </ul>
  <h3>Open Loop Equations of Motion</h3>
  <p>$$\dot{\mathbf{x}} = \mathbf{f}_o(\mathbf{x}, \mathbf{r}_c, \mathbf{r}_k, \mathbf{p}_k, t)$$</p>
</div>

# Lumped Passive+Active Controller

- True human controller is practically impossible to isolate and identify
- Identify a controller for a similar system that causes the same behavior as the real system

## Simple State Feedback

$$\mathbf{r}_c(t) = -\mathbf{K}\mathbf{x}(t)$$

## Unknown Parameters

$$\mathbf{p}_u = \mathrm{vec}(\mathbf{K})$$

## Closed Loop Equations of Motion

$$\dot{\mathbf{x}} = \mathbf{f}_c(\mathbf{x}, \mathbf{r}_k, \mathbf{p}_k, \mathbf{p}_u, t)$$

# Generate Data

## Specify the psuedo-random platform acceleration
 
$$a(t)=\sum_{i=1}^{12} A_i\sin(\omega_i t)$$

$\omega_i$ spans human operating bandwidth: 0.15 rad/s to 15.0 rad/s.

## Choose a stable controller

$$
\mathbf{K} =
\begin{bmatrix}
  950 & 175 & 185 & 50 \\
  45 &  290 & 60 & 26
\end{bmatrix}
$$

## Simulate closed loop system under the influence of perturbations
 
$$ \mathbf{x} = \int_{t_0}^{t_f} \mathbf{f}_c(\mathbf{x}, \mathbf{r}_k, \mathbf{p}_k, \mathbf{p}_u, t) dt $$

## Add Gaussian measurement noise

$\mathbf{x}_m(t) = \mathbf{x}(t) + \mathbf{v}_x(t)$

$a_m(t) = a(t) + v_a(t)$

# Parameter Identification Problem Specification

Given noisy measurements of the states, $\mathbf{x}_m$, and the platform acceleration can we identify the controller parameters $\mathbf{K}$?

Objective:

$$ J(\mathbf{p}_u) = \int [\mathbf{x}_m(t) - \mathbf{x}(t)]^2 dt$$

subject to:

$$ \mathbf{x}(t) = \int \mathbf{f}_c(\mathbf{x}, a_m, \mathbf{p}_k, \mathbf{p}_u, t) dt $$

The NLP objective is then:

$$J(\mathbf{\theta}) = \sum_{i=1}^N h [\mathbf{x}_{mi} - \mathbf{x}_i]^2$$

where

$$ \mathbf{\theta} = [\mathbf{x}, \mathbf{p}_u] $$

Subject to the constraints:

$$\mathbf{g}(\mathbf{\theta}) = 0 = \mathbf{f}_c(\mathbf{x}, a_{m}, \mathbf{p}_u)$$

And the initial guess:

$$\mathbf{\theta}_0 = [\mathbf{x}_m, \mathbf{0}]$$

For, $N$ = 6000:

- 24012 free variables
- 384000 no-zero entries in the Jacobian

# Results

Converges in 11 iterations in 2.8 seconds of computation time.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Known</th>
      <th>Identified</th>
      <th>Error</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>$k_{00}$</th>
      <td>950</td>
      <td>946</td>
      <td>-0.4%</td>
    </tr>
    <tr>
      <th>$k_{01}$</th>
      <td>175</td>
      <td>177</td>
      <td>1.4%</td>
    </tr>
    <tr>
      <th>$k_{02}$</th>
      <td>185</td>
      <td>185</td>
      <td>-0.2%</td>
    </tr>
    <tr>
      <th>$k_{03}$</th>
      <td>50</td>
      <td>55</td>
      <td>9.4%</td>
    </tr>
    <tr>
      <th>$k_{10}$</th>
      <td>45</td>
      <td>45</td>
      <td>1.1%</td>
    </tr>
    <tr>
      <th>$k_{11}$</th>
      <td>290</td>
      <td>289</td>
      <td>-0.3%</td>
    </tr>
    <tr>
      <th>$k_{12}$</th>
      <td>60</td>
      <td>59</td>
      <td>-2.1%</td>
    </tr>
    <tr>
      <th>$k_{13}$</th>
      <td>26</td>
      <td>27</td>
      <td>4.2%</td>
    </tr>
  </tbody>
</table>


# Identified State Trajectories

![](trajectory-comparison.png)

# Conclusion

- Direct collocation is suitable for biomechanical parameter identification
- Computation speeds are orders of magnitude faster than shooting
- Parameter identification accuracy improves with # nodes
- Complex problems can be solved with few lines of code and high level mathematical abstractions